<a href="https://colab.research.google.com/github/RMoulla/MLB_Mai/blob/main/TP_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exploration et Entraînement d'Embeddings Word2Vec avec Gensim**
Dans ce TP, nous allons explorer les embeddings de mots en deux étapes. Dans un premier temps, nous utiliserons un modèle Word2Vec pré-entraîné afin de comprendre comment les relations sémantiques entre les mots sont capturées. Nous analyserons des similarités, des analogies, et visualiserons ces relations dans l'espace vectoriel. Ensuite, dans la seconde partie, nous allons entraîner notre propre modèle Word2Vec sur le corpus Reuters, un ensemble d'articles économiques et financiers. L'objectif est de comprendre le processus d'entraînement d'un modèle Word2Vec et de comparer les relations sémantiques et analogiques obtenues avec un corpus spécifique. À travers ces deux étapes, nous développerons une compréhension pratique des embeddings de mots et de leur utilité dans des tâches de traitement du langage naturel.

## Outils :
* **Python**
* **Gensim** pour Word2Vec
* **NLTK** pour le corpus Reuters
* **Matplotlib** pour les visualisations

---

## **Partie 1 : Explorer un modèle Word2Vec pré-entraîné (Gensim)**
Dans cette partie, nous allons utiliser un modèle Word2Vec déjà pré-entraîné pour explorer des similarités et des relations sémantiques entre les mots.

**Étape 1** : Charger un modèle Word2Vec pré-entraîné
Nous allons utiliser le modèle Google News Word2Vec, qui a été entraîné sur une vaste collection de textes issus de Google News (environ 100 milliards de mots).

In [ ]:
!pip uninstall -y numpy gensim
!pip install gensim --upgrade
!pip install numpy==1.23.5 --force-reinstall

Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
  U

In [ ]:
import gensim.downloader as api

# Charger le modèle pré-entraîné Google News Word2Vec
model = api.load('word2vec-google-news-300')

# Vérifier la taille du vocabulaire
print(f"Vocabulaire du modèle : {len(model.key_to_index)} mots")


**Étape 2** : Explorer des similarités entre les mots

In [ ]:
# Trouver les mots les plus similaires à "king"
similar_words = model.most_similar('king')
print(similar_words)


In [ ]:
# Effectuer des analogies
analogy = model.most_similar(positive=['woman', 'king'], negative=['man'])
print(analogy)


In [ ]:
analogy = model.most_similar(positive=['woman', 'doctor'], negative=['man'])
print(analogy)

**Étape 3** : Visualisation des embeddings

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Sélectionner des mots pour visualiser leurs embeddings
words = ['king', 'queen', 'man', 'woman', 'paris', 'france', 'london', 'england']
embeddings = [model[word] for word in words]

# Réduction de dimension à 2D avec PCA
pca = PCA(n_components=2)
result = pca.fit_transform(embeddings)

# Visualisation des points
plt.scatter(result[:, 0], result[:, 1])
for i, word in enumerate(words):
    plt.annotate(word, xy=(result[i, 0], result[i, 1]))
plt.show()


## **Partie 2 : Entraîner un modèle Word2Vec sur le corpus Reuters**

Dans cette partie, vous allez entraîner votre propre modèle Word2Vec en utilisant le corpus Reuters disponible dans NLTK.

**Étape 1** : Chargement des données.


In [ ]:
import nltk
from nltk.corpus import reuters

# Télécharger et charger le corpus
nltk.download('reuters')

# Exemple de texte du corpus
corpus = [reuters.raw(fileid) for fileid in reuters.fileids()]

# Visualiser un text
print(corpus[0])

[nltk_data] Downloading package reuters to /root/nltk_data...


ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT
  Mounting trade friction between the
  U.S. And Japan has raised fears among many of Asia's exporting
  nations that the row could inflict far-reaching economic
  damage, businessmen and officials said.
      They told Reuter correspondents in Asian capitals a U.S.
  Move against Japan might boost protectionist sentiment in the
  U.S. And lead to curbs on American imports of their products.
      But some exporters said that while the conflict would hurt
  them in the long-run, in the short-term Tokyo's loss might be
  their gain.
      The U.S. Has said it will impose 300 mln dlrs of tariffs on
  imports of Japanese electronics goods on April 17, in
  retaliation for Japan's alleged failure to stick to a pact not
  to sell semiconductors on world markets at below cost.
      Unofficial Japanese estimates put the impact of the tariffs
  at 10 billion dlrs and spokesmen for major electronics firms
  said they would virtually halt exports

## Etape 2 : Prétraiter les données.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))

# Prétraitement des documents
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

processed_corpus = [preprocess_text(text) for text in corpus]

# Compter le nombre total de tokens dans tout le corpus
total_tokens = sum(len(tokens) for tokens in processed_corpus)

# Afficher le nombre total de tokens
print(f"Nombre total de tokens dans le corpus Reuters : {total_tokens}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Nombre total de tokens dans le corpus Reuters : 839812


**Etape 3** : Entraîner un modèle Word2Vec.

In [ ]:
from gensim.models import Word2Vec

# Entraînement du modèle Word2Vec
model = Word2Vec(processed_corpus, vector_size=100, window=5, min_count=5, sg=0)

**Etape 4** : Trouver des analogies sémantiques.

In [ ]:
similar_words = model.wv.most_similar('energy')
print(similar_words)

[('petroleum', 0.7597895860671997), ('almir', 0.742469072341919), ('arturo', 0.7198929786682129), ('apea', 0.7046532034873962), ('valero', 0.7012474536895752), ('masse', 0.6994137167930603), ('fernando', 0.6922779083251953), ('hernandez', 0.6842980980873108), ('santos', 0.6823518872261047), ('commerce', 0.6551962494850159)]
